## IMPORTACIONES

In [3]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
import tensorflow as tf
from datetime import timedelta
import pickle
import matplotlib.pyplot as plt
from streamlit_echarts import st_echarts
import streamlit as st


## FUNCIONES

In [3]:
def validar_y_actualizar(fila):
    if  fila["cantidad_unid"] >= 1:
        if (fila['id_item'] == 13887): #JERINGA MEGA INSUL 1MLx29Gx1/2x100
          fila["cantidad_frac"] += 100 * int(fila["cantidad_unid"])
          #datos = datos = datos.rename(columns={'cantidad_unid': 'cantidad_frac'})
          #fila["cantidad_unid"] = 0
        elif fila['id_item'] in {90765, 79680, 27112, 1669, 101609,}: #x'unidad  
          fila["cantidad_frac"] += int(fila["cantidad_unid"])

        elif(fila['id_item'] == 54122): #XARELTO COM-RECx10MGx10
          fila["cantidad_frac"] += 10 * int(fila["cantidad_unid"])
        
        elif(fila['id_item'] == 88275): #MICARDIX
           fila['cantidad_frac'] += 28 * int(fila["cantidad_unid"])  
    return fila


# Preparar los datos
def preparar_datos(datos):
    # Seleccionar columnas específicas
    columnas_especificas = ['Fecha', 'id_item', 'cantidad_unid', 'cantidad_frac']
    datos = datos[columnas_especificas]

    # Aplicar la función a cada fila
    datos = datos.apply(validar_y_actualizar, axis=1)
    datos = datos.drop(columns=["cantidad_unid"])

    print(datos)
    # Convertir la columna Fecha a formato datetime
    datos['Fecha'] = pd.to_datetime(datos['Fecha'], format='%d/%m/%Y %H:%M')
    # Establecer la hora y el minuto a 0
    datos['Fecha'] = datos['Fecha'].apply(lambda dt: dt.replace(hour=0, minute=0, second=0))

    # Agrupar por Fecha e id_item y sumar cantidad_frac
    datos = datos.groupby(['Fecha', 'id_item'], as_index=False).sum()

    # Ordenar el dataset por Fecha
    datos.sort_index(inplace=False)
    datos = datos.set_index('Fecha')

    return datos

# Definir la función de pérdida RMSE
def root_mean_squared_error(y_true, y_pred):
    return tf.math.sqrt(tf.math.reduce_mean(tf.square(y_pred - y_true)))

# Cargar el modelo y el escalador guardados
def cargar_modelo_y_scaler(id_item):
    modelo_path = f'../data/modelo_{id_item}.keras'
    scaler_path = f'../data/scaler_{id_item}.pkl'

    with open(scaler_path, 'rb') as f:
        scaler = pickle.load(f)
    modelo = load_model(modelo_path, custom_objects={'root_mean_squared_error': root_mean_squared_error})

    return modelo, scaler

# Predecir futuros valores
def predecir(x, model, scaler):
    y_pred_s = model.predict(x, verbose=0)
    y_pred = scaler.inverse_transform(y_pred_s)
    return y_pred.flatten()

# Reinterpolar los datos para cada id_item sin cambiar el índice
def reinterpolar_datos_por_id(datos):
    datos_reinterpolados = pd.DataFrame()

    for id_item in datos['id_item'].unique():
        df_item = datos[datos['id_item'] == id_item].copy()

        # Reinterpolar con frecuencia diaria
        df_item = df_item.asfreq(freq='D', fill_value=0)

        # Volver a agregar el id_item
        df_item['id_item'] = id_item

        # Concatenar los resultados
        datos_reinterpolados = pd.concat([datos_reinterpolados, df_item])

    return datos_reinterpolados.reset_index()

def generar_predicciones_futuras(df, id_item, input_length, num_predicciones):
    modelo, scaler = cargar_modelo_y_scaler(id_item)

    ultima_fecha = df['Fecha'].iloc[-1] # Access the last date from the 'Fecha' column
    print(ultima_fecha)
    fechas_futuras = [ultima_fecha + timedelta(days=i) for i in range(1, num_predicciones + 1)]

    ultimo_segmento = df['cantidad_frac'][-input_length:].values
    ultimo_segmento = ultimo_segmento.reshape((1, input_length, 1))

    predicciones_futuras = []
    segmento_actual = ultimo_segmento

    for _ in range(num_predicciones):
        prediccion = predecir(segmento_actual, modelo, scaler)
        predicciones_futuras.append(prediccion[0])

        nuevo_valor = np.array(prediccion[0]).reshape(1, 1, 1)
        segmento_actual = np.append(segmento_actual[:, 1:, :], nuevo_valor, axis=1)

    # Crear un DataFrame con las predicciones futuras, fechas y el id_item correspondiente
    resultados_futuros = pd.DataFrame({
        'Fecha': fechas_futuras,
        'Predicción': predicciones_futuras,
        'id_item': id_item  # Agregar el id_item correspondiente
    })

    return resultados_futuros



# Ejecutar todo el proceso para todos los id_item
def predecir_para_todos_los_items(datos, input_length, num_predicciones):
    resultados_totales = pd.DataFrame()
    lista = [90765, 27112, 13887, 90765, 79680, 1669, 101609, 54122, 88275]
    for id_item in lista:
        # Filtrar los datos para el id_item actual
        df_item = datos[datos['id_item'] == id_item]
        # Generar predicciones para este id_item
        resultados_item = generar_predicciones_futuras(df_item, id_item, input_length, num_predicciones)
        # Concatenar los resultados al DataFrame total
        resultados_totales = pd.concat([resultados_totales, resultados_item])

    return resultados_totales




## MAIN

In [4]:
def main():
    st.title('Predicción de Ventas')

    uploaded_file = st.file_uploader("Sube tu archivo CSV", type="csv")
    if uploaded_file is not None:
        datos = pd.read_csv(uploaded_file, delimiter=';')
        datos = preparar_datos(datos)
        datos = reinterpolar_datos_por_id(datos)
        resultados_futuros = predecir_para_todos_los_items(datos, 24, 4)
        
        st.write("Resultados de Predicción:")
        st.dataframe(resultados_futuros)
        lista = [90765, 27112, 13887, 90765, 79680, 1669, 101609, 54122, 88275]

        #Graficar ventas promedio
        for id_item in lista:
            plot_ventas_promedios(datos, id_item)

        # Graficar para cada id_item
        
        for id_item in lista:
            df_item = datos[datos['id_item'] == id_item]
            predicciones_item = resultados_futuros[resultados_futuros['id_item'] == id_item]
            st.write(f"Gráfico para id_item {id_item}")
            plot_predicciones_vs_realidad(df_item, predicciones_item, id_item)


In [11]:
main()

2024-08-18 23:11:00.316 
  command:

    streamlit run c:\Users\jeanf\miniconda3\envs\streamlit\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


## GRÁFICOS

In [5]:
def plot_ventas_promedios(datos, id_item):
    datos[datos['id_item'] == id_item].inplace = True

    axis = datos.groupby(datos['Fecha'].dt.month)[['cantidad_frac']].mean().plot(figsize = (10,5), marker = 'o', color='r')
    axis.set_title(f'Ventas promedio por Mes {id_item}')

    axis = datos.groupby(datos['Fecha'].dt.year)[['cantidad_frac']].mean().plot(figsize = (10,5), marker = 'o', color='r')
    axis.set_title(f'Ventas promedio por Año {id_item}')

    axis = datos.groupby(datos['Fecha'].dt.day)[['cantidad_frac']].mean().plot(figsize = (10,5), marker = 'o', color='r')
    axis.set_title(f'Ventas promedio por Día {id_item}')

In [6]:
def plot_predicciones_vs_realidad(df, predicciones, id_item):
    plt.figure(figsize=(10, 5))
    plt.plot(df.index, df['cantidad_frac'], label='Cantidad Real')
    plt.plot(predicciones['Fecha'], predicciones['Predicción'], label='Predicción', linestyle='--')
    plt.title(f'Predicciones vs Realidad para id_item {id_item}')
    plt.xlabel('Fecha')
    plt.ylabel('Cantidad')
    plt.legend()
    plt.grid(True)
    st.pyplot(plt)


## TEST

In [1]:
provincias_y_ciudades = {
    'Guayas': ['crnel. marcelino maridueña', 'alfredo baquerizo moreno (jujan)', 'duran', 'gnral. antonio elizalde', 'lomas de sargentillo', 'playas', 'samborondon', 'santa lucia', 'simon bolivar', 'Guayaquil', 'Durán', 'Daule', 'Milagro', 'Samborondón', 'Velasco Ibarra', 'Naranjal', 'Balzar', 'El Triunfo', 'Yaguachi', 'Pedro Carbo', 'Salitre', 'Alfredo Baquerizo Moreno', 'Naranjito', 'San Miguel', 'Píllaro', 'Simón Bolívar', 'Santa Lucía', 'Balao', 'Palestina', 'Narcisa de Jesús', 'Colimes', 'El Empalme', 'Isidro Ayora', 'Nobol', 'San Jacinto de Yaguachi'],
    'Pichincha': ['mejia', 'Quito', 'Sangolquí', 'Cayambe', 'Pedro Vicente Maldonado', 'Puerto Quito', 'Machachi', 'Tabacundo', 'Mejía', 'Pedro Moncayo', 'Rumiñahui', 'San Miguel de Los Bancos'],
    'Azuay': ['camilo ponce enriquez', 'giron', 'Cuenca', 'Gualaceo', 'Paute', 'Chordeleg', 'Camilo Ponce Enríquez', 'Nabón', 'Guachapala', 'Pucará', 'San Fernando', 'Sevilla de Oro', 'El Pan', 'Girón', 'Oña', 'Las Lajas', 'Santa Isabel'],
    'Sto. Domingo': ['santo domingo de los tsachilas', 'Santo Domingo', 'La Concordia'],
    'El Oro': ['marcabeli', 'Machala', 'Pasaje', 'Santa Rosa', 'Huaquillas', 'Piñas', 'El Guabo', 'Arenillas', 'Balsas', 'Marcabelí', 'Portovelo', 'Zaruma', 'Las Naves', 'Atahualpa', 'Chilla'],
    'Manabí': ['jaramijo', 'junin', 'pajan', 'puerto lopez', 'Manta', 'Portoviejo', 'Montecristi', 'Chone', 'El Carmen', 'Jipijapa', 'Rocafuerte', 'Santa Ana', 'San Vicente', 'Olmedo', 'Paján', 'Sucre', 'Bahía de Caráquez', 'Jaramijó', 'Pedernales', 'Puerto López', 'Jama', '24 de Mayo', 'Bolívar', 'Chinchipe', 'Flavio Alfaro', 'Junín', 'Pichincha', 'Paján', 'Puerto López', 'Tosagua'],
    'Loja': ['espindola', 'gonzanama', 'macara', 'saraguro', 'sigsig', 'Loja', 'Catamayo', 'Macará', 'Cariamanga', 'Zapotillo', 'Celica', 'Chaguarpamba', 'Pindal', 'Alamor', 'Quilanga', 'Sozoranga', 'Calvas', 'Espíndola', 'Gonzanamá', 'Paltas', 'Puyango'],
    'Los Ríos': ['urdaneta', 'Quevedo', 'Babahoyo', 'Ventanas', 'Buena Fe', 'Vinces', 'Valencia', 'Montalvo', 'Mocache', 'Palenque', 'Catarama', 'Quinsaloma', 'Puebloviejo', 'Baba'],
    'Tungurahua': ['santiago de pillaro', 'Ambato', 'Baños de Agua Santa', 'Pelileo', 'Cevallos', 'Mocha', 'Patate', 'Quero', 'San Pedro de Pelileo', 'Santiago de Píllaro', 'Tisaleo'],
    'Chimborazo': ['cumanda', 'alausi', 'Riobamba', 'Guamote', 'Chambo', 'Chunchi', 'Guano', 'Pallatanga', 'Penipe', 'Cumandá', 'Alausí', 'Colta'],
    'Esmeraldas': ['quininde', 'Esmeraldas', 'Rosa Zárate', 'San Lorenzo', 'Atacames', 'Muisne', 'Rioverde', 'Eloy Alfaro', 'Quinindé'],
    'Cotopaxi': ['la mana', 'pujili', 'saquisili', 'Latacunga', 'La Maná', 'Pujilí', 'Sigchos', 'Saquisilí', 'Pangua', 'Salcedo'],
    'Imbabura': ['san miguel de urcuqui', 'Ibarra', 'Otavalo', 'Cotacachi', 'Pimampiro', 'Urcuquí', 'Atuntaqui', 'Antonio Ante', 'San Miguel de Urcuquí'],
    'Santa Elena': ['La Libertad', 'Santa Elena', 'Salinas', 'Salinas (Salinas, Cab. Cantonal)'],
    'Carchi': ['bolivar', 'montufar', 'tulcan', 'Tulcán', 'San Gabriel', 'Huaca', 'Bolívar', 'Mira', 'Espejo', 'Montúfar', 'San Pedro de Huaca'],
    'Sucumbíos': ['sucumbios', 'Nueva Loja', 'Shushufindi', 'Cascales', 'Cuyabeno', 'Gonzalo Pizarro', 'Putumayo', 'Sucumbíos', 'Lago Agrio'],
    'Pastaza': ['pastaza', 'Puyo', 'Mera', 'Santa Clara', 'Arajuno'],
    'Orellana': ['El Coca', 'La Joya de los Sachas', 'Aguarico', 'Loreto', 'Puerto Francisco de Orellana'],
    'Morona Santiago': ['limon indanza', 'sucua', 'Macas', 'Gualaquiza', 'Morona', 'Palora', 'Sucúa', 'Logroño', 'Santiago', 'Taisha', 'Huamboya', 'San Juan Bosco', 'Limón Indanza', 'Pablo Sexto', 'Tiwintza'],
    'Zamora Chinchipe': ['centinela del condor', 'Zamora', 'Yantzaza', 'El Pangui', 'Yacuambi', 'Centinela del Cóndor', 'Nangaritza', 'Paquisha', 'Chinchipe', 'Palanda'],
    'Cañar': ['biblian', 'deleg', 'nabon', 'pucara', 'Azogues', 'La Troncal', 'Biblián', 'Déleg', 'Suscal', 'Biblían', 'Cañar', 'El Tambo'],
    'Napo': ['Tena', 'Archidona', 'Carlos Julio Arosemena Tola', 'El Chaco', 'Quijos'],
    'Bolívar': ['echeandia', 'Guaranda', 'Chillanes', 'Echeandía', 'San Miguel', 'Chimbo', 'Caluma', 'Echeandía'],
    'Galápagos': ['san cristobal', 'Puerto Baquerizo Moreno', 'Puerto Ayora', 'Puerto Villamil', 'Isabela', 'San Cristóbal', 'Santa Cruz'],
}

In [12]:
import streamlit as st
import folium
import geopandas as gpd
import pandas as pd
from streamlit_folium import st_folium



# Convertir el diccionario a un DataFrame
data = {'Provincia': list(provincias_y_ciudades.keys()), 
        'Valor': [len(ciudades) for ciudades in provincias_y_ciudades.values()]}

df = pd.DataFrame(data)

# Cargar el archivo GeoJSON
geojson_file = 'provinces.geojson'
gdf = gpd.read_file(geojson_file)

# Crear un mapa base centrado en Ecuador
m = folium.Map(location=[-1.8312, -78.1834], zoom_start=6)

# Agregar el choropleth map
folium.Choropleth(
    geo_data=gdf,
    name='choropleth',
    data=df,
    columns=['Provincia', 'Valor'],
    key_on='feature.properties.province',  # Ajustado según el campo correcto en tu GeoJSON
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Cantidad de Ciudades por Provincia'
).add_to(m)

# Mostrar el mapa en Streamlit
st.title('Choropleth Map de Ecuador')
st.write('Este es un ejemplo de un Choropleth Map que muestra la cantidad de ciudades por provincia en Ecuador.')

# Renderizar el mapa en la aplicación Streamlit
st_folium(m, width=700, height=500)


2024-08-19 02:47:33.553 
  command:

    streamlit run c:\Users\jeanf\miniconda3\envs\streamlit\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


{'last_clicked': None,
 'last_object_clicked': None,
 'last_object_clicked_tooltip': None,
 'last_object_clicked_popup': None,
 'all_drawings': None,
 'last_active_drawing': None,
 'bounds': {'_southWest': {'lat': -5.015, 'lng': -92.008},
  '_northEast': {'lat': 1.681, 'lng': -75.193}},
 'zoom': 6,
 'last_circle_radius': None,
 'last_circle_polygon': None}

In [10]:
gdf = gpd.read_file(geojson_file)
print(gdf.head())  # Mostrar las primeras filas para inspeccionar las propiedades

   country    province iso_1 iso_2  \
0  Ecuador       Azuay    EC  EC-A   
1  Ecuador     Bolívar    EC  EC-B   
2  Ecuador      Carchi    EC  EC-C   
3  Ecuador       Cañar    EC  EC-F   
4  Ecuador  Chimborazo    EC  EC-H   

                                            geometry  
0  POLYGON ((-79.76400 -3.06200, -79.76100 -3.063...  
1  POLYGON ((-79.28700 -1.20700, -79.28100 -1.207...  
2  POLYGON ((-78.51000 1.18900, -78.50800 1.19200...  
3  POLYGON ((-79.35900 -2.36300, -79.35900 -2.367...  
4  POLYGON ((-79.12900 -2.20300, -79.12800 -2.203...  


In [1]:
from datetime import datetime, timedelta
 
# Create a datetime slider with a range of one week
start_date = datetime(2020, 1, 1)
end_date = start_date + timedelta(weeks=1)

In [2]:
print(start_date) # Mostrar

2020-01-01 00:00:00
